# Legendre 2D tensor product basis tests

In [1]:
import numpy as nm
import sys
sys.path.append("/home/tomas/Python_projects/sfepy")

In [2]:
from sfepy.base.base import Struct
from dg_basis import iter_by_order, LegendrePolySpace, LegendreSimplexPolySpace, LegendreTensorProductPolySpace

/home/tomas/Python_projects/sfepy/sfepy/linalg/utils.py:10: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import matrix_multiply


In [ ]:
def iter_by_order(order, dim):
    """
    Iterates over all combinations of basis functions indexes
    needed to create multidimensional basis in a way that creates hierarchical basis
    :param order: desired order of multidimensional basis
    :param dim: dimension of the basis
    :yields: tuple containing indexes, use in combine_polyvals and combine_polyvals_der
    :return: None
    """

    # nth(iter(map(lambda x: x + (order - reduce(add,x),)), range(order)), dim)
    # nth(dim, iterate(map(lambda x: x + (order - reduce(add,x),)), map(tuple, range(order))))
    # nth(2, iterate(map(lambda x: x + (order - reduce(add,x),)), map(lambda x: (x,), range(order))))
    porder = order + 1
    if dim == 1:
        for i in range(porder):
            yield (i,)
        return
    elif dim == 2:
        for k in range(porder):
            for i in range(k + 1):
                yield (i, k - i)
        return
    elif dim == 3:
        for k in range(porder):
            for j in range(k + 1):
                for i in range(j + 1):
                    yield (i, j - i, k - j)
        return

In [3]:
order = 5

In [9]:
# define 1D basis polynoms, so far only few of them
var("x")
P = [1,
     2 * x - 1,
     6 * x ** 2 - 6 * x + 1,
     20 * x ** 3 - 30 * x ** 2 + 12 * x - 1,
     70 * x ** 4 - 140 * x ** 3 + 90 * x ** 2 - 20 * x + 1,
     252 * x ** 5 - 630 * x ** 4 + 560 * x ** 3 - 210 * x ** 2 + 30 * x - 1]

In [5]:
geometry = Struct(n_vertex=2,
                  dim=1,
                  coors=nm.array([0, 1]))
order = 4
ts = LegendreTensorProductPolySpace('legb', geometry, order)
ls = LegendrePolySpace('legb', geometry, order)

In [11]:
ls.get_nth_fun(4)(x)

70*x^4 - 140*x^3 + 90*x^2 - 20*x + 1

In [10]:
var("y")
tensorP = []
for m, idx in enumerate(iter_by_order(order, 2)):
    #print("P_{} = ({})*({})".format(m, P[idx[0]], P[idx[1]].subs(x, y)))
    print("P_{} = {}".format(m, P[idx[0]]*P[idx[1]].subs(x=y)))
    tensorP.append(P[idx[0]]*P[idx[1]].subs(x=y))

P_0 = 1
P_1 = 2*x - 1
P_2 = 6*x^2 - 6*x + 1
P_3 = 20*x^3 - 30*x^2 + 12*x - 1
P_4 = 70*x^4 - 140*x^3 + 90*x^2 - 20*x + 1
P_5 = 2*y - 1
P_6 = (2*x - 1)*(2*y - 1)
P_7 = (6*x^2 - 6*x + 1)*(2*y - 1)
P_8 = (20*x^3 - 30*x^2 + 12*x - 1)*(2*y - 1)
P_9 = 6*y^2 - 6*y + 1
P_10 = (6*y^2 - 6*y + 1)*(2*x - 1)
P_11 = (6*x^2 - 6*x + 1)*(6*y^2 - 6*y + 1)
P_12 = 20*y^3 - 30*y^2 + 12*y - 1
P_13 = (20*y^3 - 30*y^2 + 12*y - 1)*(2*x - 1)
P_14 = 70*y^4 - 140*y^3 + 90*y^2 - 20*y + 1


In [ ]:
Np = len(tensorP)
from sympy.matrices import ones
vendM = ones(Np, Np)
for i in range(Np):
    for j in range(Np):
        vendM[i, j] = integrate(integrate(tensorP[i]*tensorP[j], (x, 0, 1)), (y, 0, 1))  # Fubini, yayy!
vendM

The basis is orthogonal!

In [ ]:
vendM.condition_number()

Condition number seems to grow with square of order, but does it matter, when the matrix is diagonal?